#### Lecture 5: Coding Exercise
• Baltimore City Crime Data: https://opendata.arcgis.com/api/v3/datasets/3eeb0a2cbae94b3e8549a8193717a9e1_0/downloads/data?format=csv&spatialRefId=2248

##### Directions
1. Specify the schema for the crime data set. 
2. Read the file using the schema definition
3. Cache the DataFrame
4. Show the count of the rows
5. Print the schema
6. Display first 5 rows
7. Answer following questions
 
##### Questions
1. What are distinct crime codes?
2. Count the number of crimes by the crime codes and order by the resulting counts in descending order
3. Which neighborhood had most crimes?
4. Which month of the year had most crimes?
5. What weapons were used? 
6. Which weapon was used the most?

#### Prework: Get the Data
Inspect location where the crime data was stored locally

In [0]:
%fs ls /FileStore/tables/Part1_Crime_data.csv

path,name,size
dbfs:/FileStore/tables/Part1_Crime_data.csv,Part1_Crime_data.csv,62763718


##### Define the File Location

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

crime_data_file = "/FileStore/tables/Part1_Crime_data.csv"

##### Inspect the data locally

In [0]:
%fs head /FileStore/tables/Part1_Crime_data.csv

[Truncated to first 65536 bytes]
﻿X,Y,RowID,CrimeDateTime,CrimeCode,Location,Description,Inside_Outside,Weapon,Post,District,Neighborhood,Latitude,Longitude,GeoLocation,Premise,VRIName,Total_Incidents
1444028.69115949,614013.478635031,1,2021/04/24 17:00:00+00,7A,6400 ROSEMONT AVE,AUTO THEFT,O,NA,425,NORTHEAST,ROSEMONT EAST,39.3517,-76.5343,"(39.3517,-76.5343)",STREET ,,1
1404650.49199384,609908.822236821,2,2021/04/24 14:22:00+00,4E,4700 REISTERSTOWN RD,COMMON ASSAULT,O,NA,613,NORTHWEST,LUCILLE PARK,39.3409,-76.6736,"(39.3409,-76.6736)",STREET ,,1
1402390.7637532,609063.088658197,3,2021/04/24 20:20:00+00,4E,3900 PENHURST AVE,COMMON ASSAULT,I,NA,625,NORTHWEST,DOLFIELD,39.3386,-76.6816,"(39.3386,-76.6816)",APT/CONDO - OCCUPIED,,1
1411763.33074933,599008.054752978,4,2021/04/24 10:35:00+00,4E,1900 WALBROOK AVE,COMMON ASSAULT,I,NA,731,WESTERN,MONDAWMIN,39.3109,-76.6486,"(39.3109,-76.6486)",ROW/TOWNHOUSE-OCC,Western,1
1407601.80571192,599575.150884168,5,2021/04/24 18:28:00+00,4E,2200 KOKO LN,COMMON ASSAULT,I,NA,731,WESTERN,PANWAY/BRADDISH AVENUE,39.3125,-76.6633,"(39.3125,-76.6633)",ROW/TOWNHOUSE-VAC,,1
1422570.661216,572827.044703951,6,2021/04/24 05:00:00+00,6C,3600 POTEE ST,LARCENY,O,NA,912,SOUTHERN,BROOKLYN,39.2389,-76.6108,"(39.2389,-76.6108)",GAS STATION ,,1
1428052.29856485,569463.653641604,7,2021/04/24 00:15:00+00,4A,3900 PASCAL AVE,AGG. ASSAULT,O,FIREARM,911,SOUTHERN,CURTIS BAY,39.2296,-76.5915,"(39.2296,-76.5915)",STREET ,,1
1412101.46466419,599373.596245653,8,2021/04/24 15:45:00+00,5A,1800 CLIFTON AVE,BURGLARY,I,NA,731,WESTERN,MONDAWMIN,39.3119,-76.6474,"(39.3119,-76.6474)",ROW/TOWNHOUSE-OCC,Western,1
1419495.76996134,603956.200738147,9,2021/04/24 21:26:00+00,4E,2900 WYMAN PKWY,COMMON ASSAULT,O,NA,511,NORTHERN,REMINGTON,39.3244,-76.6212,"(39.3244,-76.6212)",ALLEY,,1
1419636.75515836,590443.574600971,10,2021/04/24 23:30:00+00,6D,100 S Eutaw,LARCENY FROM AUTO,,NA,113,CENTRAL,DOWNTOWN WEST,39.2873,-76.6209,"(39.2873,-76.6209)",,,1
1405994.92240789,590135.560183809,11,2021/04/24 00:00:00+00,5B,0 N ROSEDALE ST,BURGLARY,I,NA,835,SOUTHWEST,CARROLL-SOUTH HILTON,39.2866,-76.6691,"(39.2866,-76.6691)",ROW/TOWNHOUSE-VAC,,1
1406682.85460988,595528.748156415,12,2021/04/24 22:13:00+00,4E,2900 WINCHESTER ST,COMMON ASSAULT,I,NA,814,SOUTHWEST,WINCHESTER,39.3014,-76.6666,"(39.3014,-76.6666)",ROW/TOWNHOUSE-OCC,,1
1406682.85460988,595528.748156415,13,2021/04/24 22:13:00+00,4E,2900 WINCHESTER ST,COMMON ASSAULT,I,NA,814,SOUTHWEST,WINCHESTER,39.3014,-76.6666,"(39.3014,-76.6666)",ROW/TOWNHOUSE-OCC,,1
1411154.46955094,588042.216757186,14,2021/04/24 15:13:00+00,4B,2200 WILKENS AVE,AGG. ASSAULT,O,KNIFE,834,SOUTHWEST,CARROLLTON RIDGE,39.2808,-76.6509,"(39.2808,-76.6509)",STREET ,Tri-District,1
1406141.06009626,604450.597023987,15,2021/04/24 00:00:00+00,6D,3500 ELLAMONT RD,LARCENY FROM AUTO,O,NA,625,NORTHWEST,ASHBURTON,39.3259,-76.6684,"(39.3259,-76.6684)",STREET ,,1
1433230.89453669,594838.023905335,16,2021/04/24 15:00:00+00,6E,HY & E MONUMENT ST,LARCENY,O,NA,333,EASTERN,ELLWOOD PARK/MONUMENT,39.2992,-76.5728,"(39.2992,-76.5728)",PARKING LOT-OUTSIDE ,,1
1419806.80445103,603993.917201812,17,2021/04/24 07:45:00+00,4E,PARK DR & ART MUSEUM DR,COMMON ASSAULT,O,NA,511,NORTHERN,JOHNS HOPKINS HOMEWOOD,39.3245,-76.6201,"(39.3245,-76.6201)",STREET ,,1
1416782.01103524,596441.774602213,18,2021/04/24 15:49:00+00,4B,1500 DRUID HILL AVE,AGG. ASSAULT,I,KNIFE,123,CENTRAL,UPTON,39.3038,-76.6309,"(39.3038,-76.6309)",ROW/TOWNHOUSE-OCC,Central,1
1406632.17263339,609406.04681885,19,2021/04/24 12:10:00+00,3CF,2700 W Cold Spring Ln,ROBBERY - COMMERCIAL,I,FIREARM,614,NORTHWEST,CENTRAL PARK HEIGHTS,39.3395,-76.6666,"(39.3395,-76.6666)",OTHER - INSIDE ,,1
1402973.09190664,612379.717570703,20,2021/04/24 23:00:00+00,7A,5200 FLORENCE AVE,AUTO THEFT,O,NA,633,NORTHWEST,ARLINGTON,39.3477,-76.6795,"(39.3477,-76.6795)",STREET ,Northwestern,1
1421399.77817938,588520.509149652,21,2021/04/24 02:30:00+00,4E,600 S CHARLES ST,COMMON ASSAULT,I,NA,941,SOUTHERN,OTTERBEIN,39.282,-76.6147,"(39.28

#### Directions

##### 1. Specify Schema

In [0]:
# Define Schema
crime_schema = StructType([StructField('X',DoubleType() ,True ),
                          StructField('Y',DoubleType() ,True ),
                          StructField('RowID',IntegerType() ,True ),
                          StructField('CrimeDateTime', StringType() ,True ),
                          StructField('CrimeCode',StringType() ,True ),
                          StructField('Location',StringType() ,True ),
                          StructField('Description',StringType() ,True ),
                          StructField('Inside_Outside',StringType() ,True ),
                          StructField('Weapon',StringType() ,True ),
                          StructField('Post',StringType() ,True ),
                          StructField('District',StringType() ,True ),
                          StructField('Neighborhood',StringType() ,True ),
                          StructField('Latitude',FloatType() ,True ),
                          StructField('Longitude',FloatType() ,True ),
                          StructField('GeoLocation',StringType() ,True ),
                          StructField('Premise',StringType() ,True ),
                          StructField('VRIName',StringType() ,True ),
                          StructField('Total_Incidents',IntegerType() ,True )])

##### 2. Read file to DataFrame 
Using the schema definition

In [0]:
crime_df = spark.read.csv(crime_data_file, header=True, schema=crime_schema)

##### 3. Cache the DataFrame

In [0]:
crime_df.cache()

Out[4]: DataFrame[X: double, Y: double, RowID: int, CrimeDateTime: string, CrimeCode: string, Location: string, Description: string, Inside_Outside: string, Weapon: string, Post: string, District: string, Neighborhood: string, Latitude: float, Longitude: float, GeoLocation: string, Premise: string, VRIName: string, Total_Incidents: int]

##### 4. Show the count of the rows

In [0]:
# Note the count number may be different than what you see here.
crime_df.count()

Out[5]: 334192

##### 5. Print the schema

In [0]:
crime_df.printSchema()

root
-- X: double (nullable = true)
-- Y: double (nullable = true)
-- RowID: integer (nullable = true)
-- CrimeDateTime: string (nullable = true)
-- CrimeCode: string (nullable = true)
-- Location: string (nullable = true)
-- Description: string (nullable = true)
-- Inside_Outside: string (nullable = true)
-- Weapon: string (nullable = true)
-- Post: string (nullable = true)
-- District: string (nullable = true)
-- Neighborhood: string (nullable = true)
-- Latitude: float (nullable = true)
-- Longitude: float (nullable = true)
-- GeoLocation: string (nullable = true)
-- Premise: string (nullable = true)
-- VRIName: string (nullable = true)
-- Total_Incidents: integer (nullable = true)

##### 6. Display first 5 rows

In [0]:
display(crime_df.limit(5))

X,Y,RowID,CrimeDateTime,CrimeCode,Location,Description,Inside_Outside,Weapon,Post,District,Neighborhood,Latitude,Longitude,GeoLocation,Premise,VRIName,Total_Incidents
1444028.69115949,614013.478635031,1,2021/04/24 17:00:00+00,7A,6400 ROSEMONT AVE,AUTO THEFT,O,NA,425,NORTHEAST,ROSEMONT EAST,39.3517,-76.5343,"(39.3517,-76.5343)",STREET,null,1
1404650.49199384,609908.822236821,2,2021/04/24 14:22:00+00,4E,4700 REISTERSTOWN RD,COMMON ASSAULT,O,NA,613,NORTHWEST,LUCILLE PARK,39.3409,-76.6736,"(39.3409,-76.6736)",STREET,null,1
1402390.7637532,609063.088658197,3,2021/04/24 20:20:00+00,4E,3900 PENHURST AVE,COMMON ASSAULT,I,NA,625,NORTHWEST,DOLFIELD,39.3386,-76.6816,"(39.3386,-76.6816)",APT/CONDO - OCCUPIED,null,1
1411763.33074933,599008.054752978,4,2021/04/24 10:35:00+00,4E,1900 WALBROOK AVE,COMMON ASSAULT,I,NA,731,WESTERN,MONDAWMIN,39.3109,-76.6486,"(39.3109,-76.6486)",ROW/TOWNHOUSE-OCC,Western,1
1407601.80571192,599575.150884168,5,2021/04/24 18:28:00+00,4E,2200 KOKO LN,COMMON ASSAULT,I,NA,731,WESTERN,PANWAY/BRADDISH AVENUE,39.3125,-76.6633,"(39.3125,-76.6633)",ROW/TOWNHOUSE-VAC,null,1


##### 7. Answer following questions using PySpark and DataFrames

#### Questions

##### 1. What are distinct crime codes?

In [0]:
# Your code here. Note: The actual output maybe different than what you see below.
crime_df.select("CrimeCode").where(col("CrimeCode").isNotNull()).distinct().show()

+---------+
CrimeCode|
+---------+
 3K|
 1A|
 4B|
 3B|
 7A|
 3NF|
 5D|
 3AF|
 3JO|
 4C|
 5A|
 6C|
 4E|
 6D|
 5B|
 4A|
 3CF|
 4D|
 6J|
 6E|
+---------+
only showing top 20 rows

##### 2. Count the number of crimes by the crime codes and order by the resulting counts in descending order

In [0]:
(crime_df
 .select("CrimeCode").where(col("CrimeCode").isNotNull())
 .groupBy("CrimeCode")
 .count()
 .orderBy("count", ascending=False)
 .withColumnRenamed('count', 'Sum_Crimes')
 .show(truncate =False))


+---------+----------+
CrimeCode|Sum_Crimes|
+---------+----------+
4E |56879 |
6D |42939 |
7A |27651 |
5A |27649 |
6J |18798 |
6E |16832 |
6G |16818 |
6C |16035 |
4C |15385 |
3AF |10855 |
5D |10642 |
4A |9653 |
4B |8377 |
3B |7542 |
9S |4718 |
4D |4669 |
5B |3829 |
5C |2753 |
3CF |2689 |
6F |2565 |
+---------+----------+
only showing top 20 rows

##### 3. Which neighborhood had most crimes?

In [0]:
# Your code here. Note: The actual output maybe different than what you see below.
(crime_df
 .select("Neighborhood").where(col("CrimeCode").isNotNull())
 .groupBy("Neighborhood")
 .count()
 .orderBy(("count"), ascending=False)
 .withColumnRenamed('count', 'Sum_Of_Crimes')
 .show(1))

+------------+-------------+
Neighborhood|Sum_Of_Crimes|
+------------+-------------+
 DOWNTOWN| 11214|
+------------+-------------+
only showing top 1 row

##### 4. Which month of the year had most crimes?

In [0]:
crime_df2 = (crime_df.withColumn("timestamp",to_timestamp(col("CrimeDateTime"), "yyyy/MM/dd HH:mm:ss+SS")))
            
(crime_df2
 .select("timestamp").where(col("timestamp").isNotNull())
 .groupBy(month("timestamp").alias("CrimeMonth"))
 .count()
 .orderBy("count", ascending=False)
 .withColumnRenamed('count', 'Sum_Of_Crimes')
 .show(1))

+----------+-------------+
CrimeMonth|Sum_Of_Crimes|
+----------+-------------+
 8| 29578|
+----------+-------------+
only showing top 1 row

##### 5. What weapons were used?

In [0]:
# Your code here. Note: The actual output maybe different than what you see below.
(crime_df.select("Weapon")
.where(col("Weapon") != "NA")
.distinct()
.show())

+-------+
 Weapon|
+-------+
 HANDS|
 KNIFE|
 OTHER|
 FIRE|
FIREARM|
+-------+

##### 6. Which weapon was used the most?

In [0]:
# Your code here. Note: The actual output maybe different than what you see below.
(crime_df.select("Weapon").where(col("Weapon") != "NA").groupBy("Weapon")
.count()
.orderBy("count", ascending = False)
.show(1))

+-------+-----+
 Weapon|count|
+-------+-----+
FIREARM|34051|
+-------+-----+
only showing top 1 row